In [1]:
%autosave 0
%matplotlib inline
import urllib.request as req
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import random, re
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

Autosave disabled


In [ ]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import Adam
from keras.utils import np_utils

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(60000, 784).astype('float32')
X_test = X_test.reshape(10000, 784).astype('float')
X_train /= 255
X_test /= 255

y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

model = Sequential()
model.add(Dense(512, input_shape=(784,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer = Adam(),
    metrics=['accuracy'])

hist = model.fit(X_train, y_train)

score = model.evaluate(X_test, y_test, verbose=1)
print('loss=', score[0])
print('accuracy=', score[1])

Using TensorFlow backend.


11493376/11490434 [==============================] - 58s    
Epoch 1/10
60000/60000 [==============================] - 25s - loss: 0.2136 - acc: 0.9347    
Epoch 2/10
60000/60000 [==============================] - 43s - loss: 0.1013 - acc: 0.9685    
Epoch 3/10
14080/60000 [======>.......................] - ETA: 34s - loss: 0.0776 - acc: 0.9758

In [20]:
from keras.callbacks import EarlyStopping

csv = pd.read_csv("bmi.csv")

csv["weight"] /= 100
csv["height"] /= 200
# X = csv[["weight", "height"]].as_metrix()

bclass = {"thin":[1,0,0], "normal":[0,1,0], "fat":[0,0,1]}
y = np.empty((20000, 3))

for i,v in enumerate(csv["label"]):
    y[i] = bclass[v]
    
X_train, y_train = X[1:15001], y[1:15001]
X_test, y_test = X[15001:20001], y[15001:20001]

model = Sequential()
model.add(Dense(512, input_shape=(2,)))
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer="rmsprop",
    metrics=['accuracy']
)

hist = model.fit(
    X_train, y_train,
    batch_size=100,
    nb_epoch=20,
    validation_split=0.1,
    callbacks=[EarlyStopping(monitor='val_loss', patience=2)],
    verbose=1
)

score = model.evaluate(X_test, y_test)
print('loss=', score[0])
print('accuracy=', score[1])

/opt/conda/lib/python3.6/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 13500 samples, validate on 1500 samples
Epoch 1/20
13500/13500 [==============================] - 1s - loss: 0.5187 - acc: 0.7924 - val_loss: 0.2731 - val_acc: 0.9093
Epoch 2/20
13500/13500 [==============================] - 1s - loss: 0.2483 - acc: 0.9005 - val_loss: 0.2220 - val_acc: 0.8887
Epoch 3/20
13500/13500 [==============================] - 1s - loss: 0.1857 - acc: 0.9227 - val_loss: 0.1462 - val_acc: 0.9347
Epoch 4/20
13500/13500 [==============================] - 1s - loss: 0.1632 - acc: 0.9287 - val_loss: 0.1201 - val_acc: 0.9500
Epoch 5/20
13500/13500 [==============================] - 1s - loss: 0.1435 - acc: 0.9387 - val_loss: 0.1103 - val_acc: 0.9520
Epoch 6/20
13500/13500 [==============================] - 1s - loss: 0.1372 - acc: 0.9423 - val_loss: 0.0759 - val_acc: 0.9860
Epoch 7/20
13500/13500 [==============================] - 1s - loss: 0.1305 - acc: 0.9426 - val_loss: 0.1195 - val_acc: 0.9387
Epoch 8/20
13500/13500 [==============================] - 1s -

In [6]:
import os, glob, json

root_dir = "../../origin/news_data"
dic_file = root_dir + "/word-dic.json"
data_file = root_dir + "/data.json"
data_file_min = root_dir + "/data-mini.json"

# 어구를 자르고 ID 로 변환하기
word_dic = { "_MAX": 0 }
def text_to_ids(text):
    text = text.strip()
    words = text.split(" ")
    result = []
    for n in words :
        n = n.strip()
        if n == "": continue
        if not n in word_dic : 
            wid = word_dic[n] = word_dic["_MAX"]
            word_dic["_MAX"] += 1
            print(wid, n)
        else :
            wid = word_dic[n]
        result.append(wid)
    return result

# 파일을 읽고 고정 길이의 배열 리턴
def file_to_ids(fname):
    with open(fname, "r") as f:
        text = f.read()
        return text_to_ids(text)
    
# 딕셔너리에 단어 모두 등록
def register_dic():
    files = glob.glob(root_dir + "/*/*.wakati", recursive=True)
    for i in files:
        file_to_ids(i)
        
def count_file_freq(fname):
    cnt = [ 0 for n in range(word_dic["_MAX"]) ]
    with open(fname, "r") as f:
        text = f.read().strip()
        ids = text_to_ids(text)
        for wid in ids:
            cnt[wid] += 1
    return cnt

# 카테고리마다 파일 읽어 들이기
def count_freq(limit = 0):
    X = []
    Y = []
    max_words = word_dic["_MAX"]
    cat_names = []
    for cat in os.listdir(root_dir):
        cat_dir = root_dir + "/" + cat
        if not os.path.isdir(cat_dir): continue
        cat_idx = len(cat_names)
        cat_names.append(cat)
        files = glob.glob(cat_dir+"/*.wakati")
        i = 0
        for path in files:
            print(path)
            cnt = count_file_freq(path)
            X.append(cnt)
            Y.append(cat_idx)
            if limit > 0:
                if i > limit: break
                i += 1
    return X, Y

# 단어 딕셔너리 만들기
if os.path.exists(dic_file) :
    word_dic = json.load(open(dic_file))
else :
    register_dic()
    json.dump(word_dic, open(dic_file, "w"))
    
# 벡터를 파일로 출력
# 테스트 목적의 소규모 데이터 만들기
X, Y = count_freq(20)
json.dump({"X": X, "Y": Y}, open(data_file_min, "w"))
# 전체 데이터를 기반으로 데이터 만들기
X, Y = count_freq()
json.dump({"X": X, "Y": Y}, open(data_file, "w"))

print("ok")

ok
